This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [1]:
%pip install boto3 python-dotenv
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
%pip install selenium chromedriver-binary


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 831.9 kB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 997.9 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 911.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 934.4 kB/s eta 0:00:000:00:01
Using cached idna-3.6-py3-none-any.whl (61 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Created wheel for chromedriver-binary: filename=chromedriver_binary-125.0.6381.0.0-py3-none-any.whl size=8014136 sha256=a51c69a650e315d4985179f3aff528d690e01acb6a8609a866dedee2610dfb09
  Stored in directory: /home/mingnh/.cache/pip/wheels/d1/7b/9b/5b924cf3aaa5a65e3fe2a

In [2]:
import os
import boto3
import glob
import time
import pathlib
from selenium import webdriver
from selenium.webdriver import ActionChains
from dotenv import load_dotenv
from zipfile import ZipFile


load_dotenv()


True

In [3]:
# The file download processing section should be placed here

#...........................................................
# Create working directories
DOWNLOAD_DIR = os.path.join(os.getcwd(), "data")
EXTRACT_DIR = os.path.join(DOWNLOAD_DIR, "extracted")
pathlib.Path(DOWNLOAD_DIR).mkdir(parents=True, exist_ok=True)
pathlib.Path(EXTRACT_DIR).mkdir(parents=True, exist_ok=True)

# Change selenium download directory
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.directory_upgrade": True,
    "download.prompt_for_download": False,
}
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", prefs)

# Start selenium
driver = webdriver.Chrome(options=options)
driver.get("https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing")

# Hover zip file to render download link
action_chains = ActionChains(driver)
action_chains.move_to_element(driver.find_element(by="css selector", value="div[data-list-item-target][role='gridcell']"))
action_chains.perform()

# select download link
download_text_element = driver.find_element(by="css selector", value="div[data-list-item-target][role='gridcell'] div[role='link'] > span")
display_text = download_text_element.get_attribute('innerText')
download_link_element = driver.find_element(by="css selector", value=f"div[data-list-item-target][role='gridcell'] div[aria-label='{download_text_element.text}'][role='link']")
download_link_element.click()

# wait until file existed in download folder
while not os.path.exists(os.path.join(DOWNLOAD_DIR, "data.zip")):
    time.sleep(2)

driver.close()


# Extract data files
with ZipFile(os.path.join(DOWNLOAD_DIR, "data.zip"), 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path=EXTRACT_DIR)

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event): operation timed out


In [5]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'demo-access-key')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'demo-secret-key')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')


In [6]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)

In [7]:
# Upload file to minio

data_paths = glob.glob(os.path.join(EXTRACT_DIR, "*.json"))
for path in data_paths:
    file_name = path.split(os.sep)[-1]
    upload_file_to_minio(path, 'data', f'data-raw/{file_name}')